In [ ]:
# Data Mining  Lab Assignment 4 
# Name - Raushan Raj
# Enrollment No - IIT2018031
# Section - "A"

In [ ]:
import numpy as np
import pandas as pd
import random
from sklearn.model_selection import train_test_split
import sys,json

In [ ]:
# To read dataset "car.data"

dataframe = pd.read_csv('/content/car.data.txt',header = None, names = ['Buying','Maintenance','Doors','Persons','Lugboot','Safety','Class'])
dataframe.head()

,Buying,Maintenance,Doors,Persons,Lugboot,Safety,Class
0,vhigh,vhigh,2,2,small,low,unacc
1,vhigh,vhigh,2,2,small,med,unacc
2,vhigh,vhigh,2,2,small,high,unacc
3,vhigh,vhigh,2,2,med,low,unacc
4,vhigh,vhigh,2,2,med,med,unacc


In [ ]:
# Function to calculate ENTROPY

def ENTROPY(df,label):
    value,count = np.unique(df[label],return_counts = True)
    LENGTH = len(count)
    return sum((-count[i]/np.sum(count))*np.log2(count[i]/np.sum(count)) for i in range(LENGTH))

In [ ]:
# Function to calculate Information Gain

def INF_GAIN(df,feature,label):
    ent = ENTROPY(df,label)
    value,count = np.unique(df[feature],return_counts = True)
    length = len(count)
    return ent-sum((count[i]/sum(count))*ENTROPY(df.where(df[feature]==value[i]).dropna(),label) for i in range(length))

In [ ]:
# Function to create Decission Tree

def CREATE_DECISSION_TREE(dataset, features, label, parent = None):
    DATA = np.unique(dataset[label],return_counts = True)
    UNIQUE = np.unique(dataset[label])

    l1 = len(UNIQUE)

    #If target label has only one value then return that value
    if l1 == 1:        
        return UNIQUE[0]
    elif len(dataset) == 0:
        return UNIQUE[np.argmax(DATA[1])]
    #If the feature set becomes empty
    elif len(features) == 0:
        return parent
    else:
        parent = UNIQUE[np.argmax(DATA[1])]
        IG = [INF_GAIN(dataset,feature,label) for feature in features]
        split_feature = features[np.argmax(IG)]
        DT = {'Split Feature':split_feature}

        for value in np.unique(dataset[split_feature]):
            new_features = [i for i in features if i != split_feature]
            new_dataset = dataset.where(dataset[split_feature] == value).dropna()
            new_tree = CREATE_DECISSION_TREE(new_dataset, new_features, label, parent)
            DT[value] = new_tree
    return DT

In [ ]:
# Function to generate Decission Rules

def Generate_Rules(decisiontree, rules, lst = [], split_feature = None):
    split_feature = decisiontree['Split Feature']
    for x in decisiontree:
        new_list = list(lst)
        if x!= 'Split Feature':
            if type(decisiontree[x]) == dict:
                new_list.append((split_feature, x))
                Generate_Rules(decisiontree[x],rules,new_list,split_feature)
            else:
                new_list.append((split_feature,x))
                if decisiontree[x] not in rules:
                    rules[decisiontree[x]] = set()
                rules[decisiontree[x]].add(tuple(new_list))

In [ ]:
# Split the Dataset into Training and Testing Data

# Training Data = 70% , Testing Data = 30%
X_train,X_test = train_test_split(dataframe,test_size=0.3)



In [ ]:
# Create the Decision Tree
features = list(dataframe.columns)
features.remove('Class')
decisiontree = CREATE_DECISSION_TREE(X_train,features,'Class')

# Generate Decision Rules
rules = {}
Generate_Rules(decisiontree,rules)

In [ ]:
# TREE GENERATED

print("DECISION TREE \n ""--------------------------------------------------------------------------..........----------------------""\n A tree node has split feature which is its name and then sub trees for different values of that feature\n""................................................................................................")
print(json.dumps(decisiontree,indent = 4))

DECISION TREE 
 --------------------------------------------------------------------------..........----------------------
 A tree node has split feature which is its name and then sub trees for different values of that feature
................................................................................................
{
    "Split Feature": "Safety",
    "high": {
        "Split Feature": "Persons",
        "2": "unacc",
        "4": {
            "Split Feature": "Buying",
            "high": {
                "Split Feature": "Maintenance",
                "high": "acc",
                "low": "acc",
                "med": "acc",
                "vhigh": "unacc"
            },
            "low": {
                "Split Feature": "Maintenance",
                "high": {
                    "Split Feature": "Lugboot",
                    "big": "vgood",
                    "med": {
                        "Split Feature": "Doors",
                        "2": "acc",
             